# RKKY energy term

In this tutorial we demonstrate how to use RKKY energy term in ubermag simulations. We start by importing the modules we are going to use. Please note that we are importing `random` as well, so we can initialise our magnetisation as a random state.

In [1]:
import random
import discretisedfield as df
import micromagneticmodel as mm
import oommfc as mc

random.seed(2)  # this way we ensure reproducibility

Our sample consists of three subregions. More precisely, two magnetic, and a non-magnetic spacer.

In [2]:
p1 = (0, 0, 0)
p2 = (60e-9, 60e-9, 22e-9)
region = df.Region(p1=p1, p2=p2)
subregions={'bottom': df.Region(p1=(0, 0, 0), p2=(60e-9, 60e-9, 10e-9)),
            'spacer': df.Region(p1=(0, 0, 10e-9), p2=(60e-9, 60e-9, 12e-9)),
            'top': df.Region(p1=(0, 0, 12e-9), p2=(60e-9, 60e-9, 22e-9))}
mesh = df.Mesh(region=region, n=(20, 20, 11), subregions=subregions)

In [3]:
mesh.mpl.subregions()

Our energy equation consists of exchange, uniaxial anisotropy and RKKY energy terms. In order to define RKKY interaction, we have to pass `sigma` and `sigma2` parameters, as well as subregions between which RKKY occurs. More precisely, by passing two subregions, two closest mutually facing surfaces are going to be identified automatically. In addition, please note that we set up norm of the field using a dictionary and the value using a `lambda` function. We could have done the same thing by writing full Python functions, but here we want to show that there are many different ways how a field can be defined.

In [4]:
system = mm.System(name='rkky')
system.energy = (mm.Exchange(A=1e-12) +
                 mm.RKKY(sigma=-1e-4, sigma2=0, subregions=['bottom', 'top']) +
                 mm.UniaxialAnisotropy(K=1e5, u=(1, 0, 0)))

norm = {'bottom': 8e6, 'top': 8e6, 'spacer': 0}
system.m = df.Field(mesh, nvdim=3, value=lambda point: [2*random.random()-1 for i in range(3)], norm=norm, valid="norm")

The initial magnetisation is:

In [5]:
system.m.sel('y').mpl(figsize=(15, 4))

And the energy equation:

In [6]:
system.energy

Now we can relax the system, and plot its magnetisation.

In [7]:
md = mc.MinDriver()
md.drive(system)

In [8]:
system.m.sel('y').mpl(figsize=(12, 3))

We can see that two layers are "antiferromagnetically coupled" because we used negative $\sigma$.